In [70]:
# earthengine-api v1.5.15 (asof 2025-07-17 )
# earthengine-api v1.5.24 (asof 2025-07-19 )
# earthengine-api v1.6.2 (asof 2025-08-04 )

import ee
import ee.batch
from pprint import pprint
from pathlib import Path
from importlib import reload
from observatorio_ipa.core import config
from observatorio_ipa.services import connections
from observatorio_ipa.services.gdrive import assets as gdrive_assets

reload(connections)

<module 'observatorio_ipa.services.connections' from 'C:\\Users\\erick\\Documents\\Projects\\teleamb\\observatorio_ipa\\src\\observatorio_ipa\\services\\connections.py'>

In [71]:
service_credentials_file = Path("../secrets/ee-observatorionieves-288939dbc1cf.json")

In [72]:
runtime_service_account = connections.GoogleServiceAccount(
    service_credentials_file.as_posix(),
)
connections.connect_to_gee(runtime_service_account)

# Connect to GDrive
gdrive_service = connections.connect_to_gdrive(runtime_service_account)

In [4]:
# from googleapiclient.discovery import build

# ... (Authentication and service object creation) ...
# service = build('drive', 'v3', credentials=creds)

results = (
    gdrive_service.files()
    .list(
        q="mimeType='application/vnd.google-apps.folder'",
        fields="nextPageToken, files(id, name, parents)",
        corpora="user",  # Or 'allDrives' if searching across shared drives
    )
    .execute()
)

items = results.get("files", [])
if not items:
    print("No folders found.")
else:
    print("Folders:")
    for item in items:
        print(f"Name: {item['name']}, ID: {item['id']}")

Folders:
Name: New_Raster_SCI_CCI, ID: 1EG0Y39mIwHnENuVj6idLCvYjRRfK1JbJ
Name: moving_company, ID: 1FnKd35DV1thb_AfnjF7-YvYnu_83IA8v


In [16]:
# Basic Export test
# A Landsat 8 surface reflectance image.
ee_test_image = ee.image.Image("LANDSAT/LC08/C02/T1_L2/LC08_044034_20210508").select(
    ["SR_B."]
)  # reflectance bands

# A region of interest.
region = ee.geometry.Geometry.BBox(-122.24, 37.13, -122.11, 37.20)

# Set the export "scale" and "crs" parameters.
task = ee.batch.Export.image.toCloudStorage(
    image=ee_test_image,
    description="test_image_export_2",
    bucket="chompitest-odes",
    region=region,
    scale=30,
    crs="EPSG:5070",
)
task.start()

In [8]:
from google.oauth2 import service_account

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [9]:

    
# Connect to Google API and build service
google_credentials = service_account.Credentials.from_service_account_info(
    runtime_service_account.credentials
)



In [2]:
from google.cloud import storage

def create_storage_client(project, credentials):
    """Creates a Google Cloud Storage client."""
    return storage.Client(project=project, credentials=credentials)

def upload_blob(storage_client, bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(
        source_file_name, if_generation_match=generation_match_precondition
    )

    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

In [10]:
storage_client = create_storage_client(runtime_service_account.project_id, google_credentials)
    
    


In [ ]:
upload_blob(
    runtime_service_account.project_id,
    google_credentials, 
    "chompitest-odes",
    "../manifests/monthly_manifest.json",
    "monthly_manifest.json",
)

In [ ]:
bucket = storage_client.bucket("chompitest-odes")
src_blob = bucket.blob("OSN/stats/stats1.csv")
src_blob.upload_from_string("This is a test file content.")

In [49]:
def _get_files(files_path: str | Path, storage_conn: storage.Client, bucket_name: str):
    files_path = Path(files_path)
    prefix = files_path.as_posix() + "/"
    len_prefix = len(prefix)
    bucket = storage_conn.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix, delimiter="/")
    list_of_files = []
    
    # Exclude Folders. Folders should end with "/" on account of delimiter="/"
    for blob in blobs:
        relative_path = blob.name[len_prefix:]
        if relative_path and "/" not in relative_path: 
            list_of_files.append(Path(relative_path))
    return list_of_files

In [50]:
files = _get_files("OSN/stats/", storage_client, "chompitest-odes")

In [51]:
files

[WindowsPath('monthly_manifest_LU20250822_LLU20250822.json'),
 WindowsPath('test_image_export_2_LU20250822_LLU20250822.tif')]

In [78]:
print(list_of_files)

[WindowsPath('OSN/stats/monthly_manifest_LU20250822_LLU20250822.json'), WindowsPath('OSN/stats/test_image_export_2_LU20250822_LLU20250822.tif')]


In [60]:
bucket.move_blob(blob='monthly_manifest_LU20250822_LLU20250822.json', new_name='OSN/stats/monthly_manifest_LU20250822_LLU20250822.json')

AttributeError: 'str' object has no attribute 'path'

In [38]:
from  datetime import date

In [ ]:
def date_stamp_file():
    

ld=date.today()
ld=ld.strftime("%Y%m%d")


#renamed_files=[{"old_name": str(f), "new_name": f"{f.stem}_LLU{ld}{f.suffix}", "renamed": False} for f in list_of_files]
renamed_files = [{"blob": blob, "old_name": str(blob.name), "new_name": str(blob.name), "renamed": True} for blob in list_of_files]
renamed_files


[{'blob': <Blob: chompitest-odes, monthly_manifest_LU20250822_LLU20250822.json, 1755901300665511>,
  'old_name': 'monthly_manifest_LU20250822_LLU20250822.json',
  'new_name': 'monthly_manifest_LU20250822_LLU20250822.json',
  'renamed': True},
 {'blob': <Blob: chompitest-odes, test_image_export_2_LU20250822_LLU20250822.tif, 1755901301731976>,
  'old_name': 'test_image_export_2_LU20250822_LLU20250822.tif',
  'new_name': 'test_image_export_2_LU20250822_LLU20250822.tif',
  'renamed': True}]

In [ ]:
for item in renamed_files:
    try:
        old_name = item["old_name"]
        new_name = item["new_name"]
        src_blob = bucket.blob(old_name)
        new_blob = bucket.rename_blob(src_blob, new_name)
        print(f"Renamed {old_name} to {new_blob.name}")
    except Exception as e:
        item["renamed"] = False

Renamed monthly_manifest_LU20250822.json to monthly_manifest_LU20250822_LLU20250822.json
Renamed test_image_export_2_LU20250822.tif to test_image_export_2_LU20250822_LLU20250822.tif


In [70]:
for item in renamed_files:
    if item['renamed']:
        source_blob = bucket.blob(item['new_name'])
        destination_blob_name = 'OSN/stats/' + item['new_name']
        new_blob = bucket.copy_blob(source_blob, bucket, destination_blob_name)
        source_blob.delete()
        print(f"Moved {item['new_name']} to OSN/stats/")

Moved monthly_manifest_LU20250822_LLU20250822.json to OSN/stats/
Moved test_image_export_2_LU20250822_LLU20250822.tif to OSN/stats/


In [62]:
import json, logging
logger = logging.getLogger(__name__)

In [64]:
def read_manifest_from_file(manifest_path: str | Path) -> dict:
    manifest_path = Path(manifest_path)
    with open(manifest_path, "r") as f:
        manifest = json.load(f)
    return manifest


def read_manifest_from_storage(
    conn: storage.Client, bucket_name: str, manifest_path: str | Path, name: str
) -> dict:
    full_path = Path(manifest_path, name)
    bucket = conn.bucket(bucket_name)
    try:
        blob = bucket.blob(full_path.as_posix())
        manifest_json = blob.download_as_text()
        return json.loads(manifest_json)
    except Exception as e:
        logger.warning(f"Error reading manifest from storage: {e}")
        return {}


def get_manifest(
    source: str,
    manifest_path: str | Path,
    manifest_name: str,
    conn: storage.Client | None,
    bucket_name: str | None,
) -> dict:
    manifest_path = Path(manifest_path)
    match source:
        case "storage":
            if not conn or not bucket_name:
                raise ValueError(
                    "Storage client and bucket name must be provided for storage source."
                )
            return read_manifest_from_storage(
                conn, bucket_name, manifest_path, manifest_name
            )

        case "file":
            return read_manifest_from_file(manifest_path / manifest_name)
        case _:
            raise ValueError(f"Unknown stats_export_target: {source}:{manifest_path}")

In [ ]:
storage_client = create_storage_client(
    runtime_service_account.project_id, google_credentials
)
bucket_name = "chompitest-odes"

In [54]:
# local Manifest
manifest = get_manifest("file", "../manifests", "monthly_manifest.json", None, None)

In [55]:
print(manifest)

{'date_created': '2025-08-22T01:19:25.499886', 'image_collection': 'projects/ee-observatorionieves/assets/Test/MODIS/Andes_MCDS4S5_Yearly_Monthly', 'images': ['Andes_MCDS4S5_Yearly_Monthly_2025_01', 'Andes_MCDS4S5_Yearly_Monthly_2025_02', 'Andes_MCDS4S5_Yearly_Monthly_2025_03', 'Andes_MCDS4S5_Yearly_Monthly_2025_04', 'Andes_MCDS4S5_Yearly_Monthly_2025_05', 'Andes_MCDS4S5_Yearly_Monthly_2025_06', 'Andes_MCDS4S5_Yearly_Monthly_2025_07']}


In [67]:
storage_manifest = get_manifest("storage", "OSN/stats/", "monthly_manifest.json", storage_client, "chompitest-odes")


In [68]:
print(storage_manifest)

{'date_created': '2025-08-22T01:19:25.499886', 'image_collection': 'projects/ee-observatorionieves/assets/Test/MODIS/Andes_MCDS4S5_Yearly_Monthly', 'images': ['Andes_MCDS4S5_Yearly_Monthly_2025_01', 'Andes_MCDS4S5_Yearly_Monthly_2025_02', 'Andes_MCDS4S5_Yearly_Monthly_2025_03', 'Andes_MCDS4S5_Yearly_Monthly_2025_04', 'Andes_MCDS4S5_Yearly_Monthly_2025_05', 'Andes_MCDS4S5_Yearly_Monthly_2025_06', 'Andes_MCDS4S5_Yearly_Monthly_2025_07']}


In [ ]:
import re
from gee_toolbox.gee import assets

def _get_imgs_in_monthly_ic(
    monthly_collection_path: str | Path, name_prefix: str
) -> list[str]:
    """Returns the names of images in a monthly collection.

    Image names are expected to have one of the following formats:
        - [name_prefix]YYYY-MM
        - [name_prefix]YYYY_MM
        (or if no name_prefix is provided)
        - *YYYY-MM
        - *YYYY_MM

    Args:
        expected_dates (list[str]): List of expected dates in the format "YYYY-MM"
        monthly_collection_path (str): Path to asset collection or folder with exported images
        name_prefix (str): Prefix of the image names

    Returns:
        list[str]: Sorted (asc) list of dates that have not been exported

    Raises:
        TypeError: If expected_dates is not a list


    """
    # monthly_collection_path will not be checked here since it's already checked to exist and
    # be a 'container' type in main process flow.
    # TODO: Replace typechecking with pydantic
    # TODO: Update typing and documentation of assets.get_asset_names
    monthly_collection_path = Path(monthly_collection_path)

    # Get names of all images already exported to assets
    exported_images = assets.list_assets(
        parent=monthly_collection_path.as_posix(), asset_types=["Image"]
    )
    exported_images = assets.get_asset_names(exported_images)
    exported_images = [Path(img).name for img in exported_images]

    # Keep only assets that start with the image prefix and end with YYYY-MM
    date_pattern = r"(\d{4})[_-](\d{2})"
    if name_prefix:
        pattern = rf"^{name_prefix}{date_pattern}$"
    else:
        pattern = rf".*{date_pattern}$"

    exported_images = [img for img in exported_images if re.fullmatch(pattern, img)]
    return exported_images


def compare_manifest_to_collection(
    manifest_src: str,
    manifest_path: str | Path,
    manifest_name: str,
    collection_path: str | Path,
    image_prefix: str,
    storage_conn: storage.Client | None = None,
    bucket_name: str | None = None,
) -> bool:

    manifest_path = Path(manifest_path)
    collection_path = Path(collection_path)

    # Get List of Images from Collection
    #image_prefix = _fix_name_prefix(image_prefix)

    collection_images = _get_imgs_in_monthly_ic(
        monthly_collection_path=collection_path,
        name_prefix=image_prefix,
    )

    # Get list of images in the manifest
    manifest = get_manifest(
        manifest_src, manifest_path, manifest_name, storage_conn, bucket_name
    )
    manifest_collection = Path(manifest.get("image_collection", ""))
    manifest_images = manifest.get("images", [])

    return (
        collection_path == manifest_collection and collection_images == manifest_images
    )

In [77]:
compare_manifest_to_collection(
    manifest_src="file",
    manifest_path="../manifests",
    manifest_name="monthly_manifest.json",
    collection_path="projects/ee-observatorionieves/assets/Test/MODIS/Andes_MCDS4S5_Yearly_Monthly",
    image_prefix="Andes_MCDS4S5_Yearly_Monthly_",
    storage_conn=None,
    bucket_name=None
)

True

In [78]:
compare_manifest_to_collection(
    manifest_src="storage",
    manifest_path="OSN/stats/",
    manifest_name="monthly_manifest.json",
    collection_path="projects/ee-observatorionieves/assets/Test/MODIS/Andes_MCDS4S5_Yearly_Monthly",
    image_prefix="Andes_MCDS4S5_Yearly_Monthly_",
    storage_conn=storage_client,
    bucket_name="chompitest-odes",
)

True

In [81]:
from datetime import datetime

In [ ]:
def create_manifest(
    images: list[str], collection_path: str | Path, meta: dict = {}
) -> str:
    collection_path = Path(collection_path)
    images.sort()

    manifest = {
        "date_created": datetime.now().isoformat(),
        "image_collection": collection_path.as_posix(),
        "first_image": images[0] if images else None,
        "last_image": images[-1] if images else None,
        "images": images,
        "metadata": meta,
    }

    manifest_json = json.dumps(manifest, indent=2)

    # manifest_path = f"{image_collection_path}/monthly_manifest.json"
    # with open(manifest_path, "w") as f:
    #     json.dump(manifest, f)
    return manifest_json


def save_manifest_to_file(
    manifest_json: str,
    manifest_path: str | Path,
    manifest_name: str,
    overwrite: bool = False,
) -> None:
    full_manifest_path = Path(manifest_path, manifest_name)
    if full_manifest_path.exists():
        if overwrite:
            full_manifest_path.unlink(missing_ok=True)
        else:
            raise FileExistsError(f"File {full_manifest_path} already exists.")
    with open(full_manifest_path, "w") as f:
        f.write(manifest_json)


def save_manifest_to_storage(
    manifest_json: str, manifest_path: str | Path, manifest_name: str, storage_conn: storage.Client, bucket_name: str, overwrite: bool = False
) -> None:
    full_path = Path(manifest_path, manifest_name)
    bucket = storage_conn.bucket(bucket_name)
    blob = bucket.blob(full_path.as_posix())

    if blob.exists():
        if overwrite:
            blob.delete()
        else:
            raise FileExistsError(f"Blob {full_path.as_posix()} already exists in bucket {bucket_name}.")
    blob.upload_from_string(manifest_json)

def save_manifest(
    target: str,
    manifest_path: str | Path,
    manifest_name: str,
    manifest_json: str,
    storage_conn: storage.Client | None,
    bucket_name: str | None,
    overwrite: bool = False,
) -> None:

    match target:
        case "storage":
            if not storage_conn or not bucket_name:
                raise ValueError(
                    "Storage client and bucket name must be provided for storage target."
                )
            return save_manifest_to_storage(
                manifest_json, manifest_path, manifest_name,  storage_conn, bucket_name, overwrite
            )

        case "file":
            return save_manifest_to_file(
                manifest_json, manifest_path, manifest_name, overwrite
            )
        case _:
            raise ValueError(f"Unknown stats_export_target: {target}:{manifest_path}")


In [100]:
monthly_collection_path="projects/ee-observatorionieves/assets/Test/MODIS/Andes_MCDS4S5_Yearly_Monthly"

images = _get_imgs_in_monthly_ic(
        monthly_collection_path=monthly_collection_path,
        name_prefix="Andes_MCDS4S5_Yearly_Monthly_",
    )
monthly_manifest = create_manifest(
    images=images,
    collection_path=monthly_collection_path,
    meta={"description": "Monthly Images"}
)


In [101]:
monthly_manifest

'{\n  "date_created": "2025-08-24T18:20:50.005101",\n  "image_collection": "projects/ee-observatorionieves/assets/Test/MODIS/Andes_MCDS4S5_Yearly_Monthly",\n  "first_image": "Andes_MCDS4S5_Yearly_Monthly_2025_01",\n  "last_image": "Andes_MCDS4S5_Yearly_Monthly_2025_07",\n  "images": [\n    "Andes_MCDS4S5_Yearly_Monthly_2025_01",\n    "Andes_MCDS4S5_Yearly_Monthly_2025_02",\n    "Andes_MCDS4S5_Yearly_Monthly_2025_03",\n    "Andes_MCDS4S5_Yearly_Monthly_2025_04",\n    "Andes_MCDS4S5_Yearly_Monthly_2025_05",\n    "Andes_MCDS4S5_Yearly_Monthly_2025_06",\n    "Andes_MCDS4S5_Yearly_Monthly_2025_07"\n  ],\n  "metadata": {\n    "description": "Monthly Images"\n  }\n}'

In [ ]:
save_manifest(
    target="file",
    manifest_path="../manifests/",
    manifest_name="monthly_manifest.json",
    manifest_json=monthly_manifest,
    storage_conn=None,
    bucket_name=None,
    overwrite=True
)

In [106]:
save_manifest(
    target="storage",
    manifest_path="OSN/stats/",
    manifest_name="monthly_manifest.json",
    manifest_json=monthly_manifest,
    storage_conn=storage_client,
    bucket_name="chompitest-odes",
    overwrite=True,
)

In [147]:
from datetime import date

In [163]:
def date_stamp_file(file: str|Path) -> Path:
    file = Path(file)
    lu=date.today().strftime("%Y%m%d")
    return file.parent / f"{file.stem}_LU{lu}{file.suffix}"

def _get_files(files_path: str | Path, storage_conn: storage.Client, bucket_name: str)-> list[Path]:
    files_path = Path(files_path)
    prefix = files_path.as_posix() + "/"
    len_prefix = len(prefix)
    bucket = storage_conn.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix, delimiter="/")
    
    
    # Exclude Folders. Folders should end with "/" on account of delimiter="/"
    list_of_files = []
    for blob in blobs:
        relative_path = blob.name[len_prefix:]
        if relative_path and "/" not in relative_path: 
            list_of_files.append(Path(relative_path))
    return list_of_files


def _rename_files(files: list[str]|list[Path], files_path: str | Path, storage_conn: storage.Client, bucket_name: str) -> list[dict]: 
    files_= [Path(f) for f in files]
    renamed_files = [{'name': f, 'new_name': date_stamp_file(f), 'succeeded': False} for f in files_]

    bucket = storage_conn.bucket(bucket_name)
    for item in renamed_files:
        try:
            full_item_name = Path(files_path, item['name']).as_posix()
            full_item_new_name = Path(files_path, item['new_name']).as_posix()
            blob = bucket.blob(full_item_name)
            new_blob = bucket.rename_blob(blob, full_item_new_name)
            item["succeeded"] = True
            print(f"Renamed {item['name']} to {new_blob.name}")
        except Exception as e:
            print(e)
            pass

    return renamed_files


def _move_files(
    files: list[str] | list[Path],
    src_path: str | Path,
    dest_path: str | Path,
    storage_conn: storage.Client,
    bucket_name: str,
    overwrite: bool = False,
):
    files_ = [{"name": Path(f), "succeeded": False} for f in files]
    src_path_ = Path(src_path)
    dest_path_ = Path(dest_path)
    bucket = storage_conn.bucket(bucket_name)
    for file in files_:
        src_file_name = Path(src_path_, file['name']).as_posix()
        dest_file_name = Path(dest_path_, file['name']).as_posix()

        src_blob = bucket.blob(src_file_name)
        dest_blob = bucket.blob(dest_file_name)
        if dest_blob.exists():
            if overwrite:
                dest_blob.delete()
            else:
                raise FileExistsError(f"File {dest_file_name} already exists in bucket {bucket_name}.")
        
        try:
            dest_blob = bucket.copy_blob(src_blob, bucket, dest_file_name)
            src_blob.delete()
            file["succeeded"] = True
            print(f"Moved {file['name']} to {dest_path_.as_posix()}/")
        except Exception as e:
            logger.warning(e)
            pass
    return files_


def _move_and_rename_files(
    files: list[str] | list[Path],
    src_path: str | Path,
    dest_path: str | Path,
    storage_conn: storage.Client,
    bucket_name: str,
    overwrite: bool = False,
):
    files_ = [{"name": Path(f), "succeeded": False} for f in files]
    src_path_ = Path(src_path)
    dest_path_ = Path(dest_path)
    bucket = storage_conn.bucket(bucket_name)
    for file in files_:
        file["new_name"] = date_stamp_file(file['name'])
        src_file_name = Path(src_path_, file['name']).as_posix()
        dest_file_name = Path(dest_path_, file["new_name"]).as_posix()

        src_blob = bucket.blob(src_file_name)
        dest_blob = bucket.blob(dest_file_name)
        if dest_blob.exists():
            if overwrite:
                dest_blob.delete()
            else:
                raise FileExistsError(f"File {dest_file_name} already exists in bucket {bucket_name}.")

        try:
            dest_blob = bucket.copy_blob(src_blob, bucket, dest_file_name)
            src_blob.delete()
            file["succeeded"] = True
            print(f"Moved and renamed {src_file_name} to {dest_file_name}")
        except Exception as e:
            logger.warning(e)
            pass
    return files_

In [135]:
storage_files = _get_files("OSN/stats/", storage_client, "chompitest-odes")
print(storage_files)

[WindowsPath('monthly_manifest.json'), WindowsPath('test_image_export.tif')]


In [ ]:
manifest_stats_files = [Path("stats1.csv")]
storage_files.extend(manifest_stats_files)

In [137]:
print(storage_files)

[WindowsPath('monthly_manifest.json'), WindowsPath('test_image_export.tif'), WindowsPath('stats1.csv')]


In [140]:
files_to_export = ["stats1.csv", "stats2.csv"]
manifest_stats_files = {"stats1.csv": "2025-08-22", "stats2.csv": "2025-08-22", "stats3.csv": "2025-08-22" }

In [143]:
# identify elements in files_to_export in storage files

found_in_storage = [Path(f) for f in files_to_export if Path(f) in storage_files]
print(found_in_storage)

[WindowsPath('stats1.csv')]


In [ ]:
renamed_files =_rename_files(found_in_storage, "OSN/stats/", storage_client, "chompitest-odes")

Renamed OSN\stats\stats1.csv to OSN/stats/stats1_LU20250824.csv


[{'name': WindowsPath('OSN/stats/stats1.csv'),
  'new_name': WindowsPath('OSN/stats/stats1_LU20250824.csv'),
  'succeeded': True}]

In [ ]:
#new_names = [item["new_name"] for item in renamed_files if item["succeeded"]]
new_names = [Path("stats1_LU20250824.csv")]

In [ ]:
moved_files =_move_and_rename_files(found_in_storage, "OSN/stats/", "OSN/stats/archived/", storage_client, "chompitest-odes", overwrite=True)

Moved and renamed OSN/stats/stats1.csv to OSN/stats/archived/stats1_LU20250824.csv


[{'name': WindowsPath('stats1.csv'),
  'succeeded': True,
  'new_name': WindowsPath('stats1_LU20250824.csv')}]

In [ ]:
successful_moves = [item for item in moved_files if item["succeeded"]]